In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore')

## データの読み込み

In [3]:
# データの読み込み
# df_org = pd.read_csv('./data/input_01.csv')
# df_org = pd.read_csv('./data/input_02.csv')
# df_org = pd.read_csv('./data/input_03.csv')
# df_org = pd.read_csv('./data/input_04.csv')
df_org = pd.read_csv('./data/input_05.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data/input_05.csv'

In [92]:
# データ量が多いのでコーディング中はサンプリング
# df = df_org.sample(100000, replace=True)
df = df_org

In [93]:
df.head(1)

,Unnamed: 0,ID,name,category,main_category,currency,deadline,goal,launched,pledged,...,Last_3_Month,Last_Year,mean_goal_in_category_last_month,median_goal_in_category_last_month,mean_goal_in_category_last_year,median_goal_Last_6_Month,mean_goal_Last_6_Month,mean_goal_Last_Week,name_len,name_number_of_word
0,0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,...,0,192,NaN,NaN,10028.166667,NaN,NaN,NaN,31.0,6


In [94]:
df.columns

Index(['Unnamed: 0', 'ID', 'name', 'category', 'main_category', 'currency',
       'deadline', 'goal', 'launched', 'pledged', 'state', 'backers',
       'country', 'usd pledged', 'usd_pledged_real', 'usd_goal_real', 'period',
       'launched_year', 'launched_month', 'launched_day', 'class',
       'name_vowel_ratio', 'n_words', 'log_goal', 'Goal_1000', 'Goal_500',
       'Goal_10', 'median_goal_Last_Week', 'Last_Week', 'Last_Month',
       'Last_6_Month', 'Last_3_Month', 'Last_Year',
       'mean_goal_in_category_last_month',
       'median_goal_in_category_last_month', 'mean_goal_in_category_last_year',
       'median_goal_Last_6_Month', 'mean_goal_Last_6_Month',
       'mean_goal_Last_Week', 'name_len', 'name_number_of_word'],
      dtype='object')

In [95]:
# df[['usd_goal_real', 'log_goal']].head()
# # 対数変換してるのに40000はさすがにおかしい
# df['log_goal'].hist()

# # この外れ値除去を行うと、最終的な精度が悪化した
# med = df['usd_goal_real'].median()
# MAD = 1.4826 * np.median(abs(df['usd_goal_real']-med))
# df = df[(med - 3 * MAD < df['usd_goal_real']) & (df['usd_goal_real'] < med + 3 * MAD)]

# df['log_goal'] = np.log(df['usd_goal_real'])

# df['usd_goal_real'].hist()

# df['log_goal'].hist()

# df['period'].hist()

In [96]:
# # ここでNaNが多いものはできれば除きたい
# df.isnull().sum()

## データの準備

In [97]:
# 数値エンコーディング（GBDT以外の場合は必須）
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df = df.apply(encoder.fit_transform)

In [98]:
df.head()

,Unnamed: 0,ID,name,category,main_category,currency,deadline,goal,launched,pledged,...,Last_3_Month,Last_Year,mean_goal_in_category_last_month,median_goal_in_category_last_month,mean_goal_in_category_last_year,median_goal_Last_6_Month,mean_goal_Last_6_Month,mean_goal_Last_Week,name_len,name_number_of_word
0,0,154087,286604,108,12,5,2285,689,217189,0,...,0,66,239,85,81,94,333,211,30,5
1,1,154088,116954,93,6,13,3039,5532,319788,8330,...,6,6,56,51,52,61,63,51,44,7
2,2,154089,314230,93,6,13,1330,5994,74361,1094,...,0,239,239,85,341,94,333,211,13,2
3,3,154090,297084,90,10,13,1014,2405,42810,1,...,1,136,239,85,164,20,8,211,48,6
4,4,154091,177276,123,7,13,2460,6124,246912,49937,...,5,241,239,85,307,75,254,211,19,2


In [99]:
y = df[['state']]
# x_cols = ['class', 'country', 'period', 'log_goal', 'launched_year', 'launched_month', 'launched_day', 
#          'name_vowel_ratio', 'Goal_1000', 'Goal_10', 'Last_Year', 'name_len', 'name_number_of_word', 
#           'mean_goal_in_category_last_month', 'mean_goal_in_category_last_year','mean_goal_Last_6_Month']
# x_cols = ['class', 'country', 'period', 'n_words', 'log_goal', 'launched_year', 'launched_month', 'launched_day', 
#          'name_vowel_ratio', 'Goal_1000', 'Goal_500', 'Goal_10', 'median_goal_Last_Week']
x_cols = ['class', 'period', 'log_goal', 'name_vowel_ratio', 'name_len', 'country', 
         'launched_day', 'launched_month', 'launched_year', 'Last_Year', 'Last_Month', 'Last_Week', 
         'mean_goal_in_category_last_year']
x = df[x_cols]

In [100]:
y.head()

,state
0,0
1,0
2,0
3,0
4,1


In [101]:
x.head()

,class,period,log_goal,name_vowel_ratio,name_len,country,launched_day,launched_month,launched_year,Last_Year,Last_Month,Last_Week,mean_goal_in_category_last_year
0,97,58,10270,398,30,9,10,7,6,66,0,0,81
1,42,59,35206,225,44,22,1,8,8,6,6,6,52
2,42,44,37850,378,13,22,11,0,4,239,0,0,341
3,78,29,19451,388,48,22,16,2,3,136,0,0,164
4,55,34,38482,261,19,22,25,1,7,241,0,0,307


## モデル学習（単体）

In [102]:
from sklearn.model_selection import train_test_split

# 訓練データとテストデータを8:2に分割
# ※評価を揃えるため df 以外の引数は変更しないこと
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state=1234)

In [103]:
import lightgbm as lgb
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score

# パラメータ
RANDOM_STATE = 10
num_round = 1000
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=TEST_SIZE,random_state=RANDOM_STATE)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

In [104]:
# LightGBMのパラメータ設定
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'binary_logloss',
    'num_leaves': 50,
    'verbose': 0
}

In [105]:
# 訓練データに対する予測
# LightGBM学習
model = lgb.train(params, lgb_train, num_round, 
                  valid_names=['train', 'valid'], valid_sets=lgb_eval,
                  early_stopping_rounds=20, verbose_eval=False)

# LightGBM推論
y_pred = model.predict(x_valid, num_iteration=model.best_iteration)

# 評価
y_pred = y_pred.round(0)
# acc_train = accuracy_score(y_test, y_pred)
acc_train = accuracy_score(y_true=y_valid, y_pred=y_pred)

print(f"訓練データの正答率: {100 * acc_train:.3f} %")    

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006003 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
訓練データの正答率: 69.541 %


In [106]:
# テストデータに対する予測
# LightGBM推論
y_pred = model.predict(x_test, num_iteration=model.best_iteration)
y_pred = y_pred.round(0)

# 評価
acc_train = accuracy_score(y_true=y_test, y_pred=y_pred)
print(f"テストデータの正答率: {100 * acc_train:.3f} %")  

# print(y_test)
# print(y_pred)

テストデータの正答率: 69.665 %


In [107]:
# feature importance for Light GBM
predictor_columns = x_train.columns
importance = pd.DataFrame(model.feature_importance(), index=predictor_columns, columns=['importance'])
# display(importance)
display(importance.sort_values('importance', ascending = False))

# predictor_columns = x_train.columns
# feat_import = list(zip(predictor_columns, list(model.feature_importances())))
# ns_df = pd.DataFrame(data = feat_import, columns=['Feat_names', 'Importance'])
# ns_df_sorted = ns_df.sort_values(['Importance', 'Feat_names'], ascending = [False, True])

# ns_df_sorted

,importance
class,2557
log_goal,1846
name_vowel_ratio,1357
name_len,1263
period,1245
launched_day,933
Last_Year,786
mean_goal_in_category_last_year,766
launched_month,668
launched_year,653


## 交差検証

In [108]:
# !pip install lightgbm

In [109]:
from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import KFold
n_split = 5 # グループ数を設定（今回は5分割）
seed = 1234
# cross_valid_mae = 0
# split_num = 1
kf = StratifiedKFold(n_splits=n_split, shuffle=True, random_state=seed)
# kf = KFold(n_splits=n_split, shuffle=True, random_state=seed)

In [110]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score
import lightgbm as lgb

# scores_train = []
scores_valid = []
models = []
itr = 1

for train_idx, valid_idx in kf.split(x, y):
    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
    x_valid, y_valid = x.iloc[valid_idx], y.iloc[valid_idx]
    
    # lightgbmのデータ構造に変換
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_valid, y_valid)
    
    # ハイパーパラメータの設定
    # metricsはaccuracy_scoreにすべきかも
    params = {'num_leaves': 64, 'objective': 'binary', 'seed': seed, 
              'verbose': 0, 'metrics': 'binary_logloss'}
    num_round = 1000

    # 学習の実行
    # カテゴリ変数を指定した場合の挙動はKaggle本 p240参照
    # categorical_features = ['category', 'main_category', 'currency', 'country']
    model = lgb.train(params, lgb_train, num_round, 
                    valid_names=['train', 'valid'], valid_sets=[lgb_train, lgb_valid],
                     early_stopping_rounds=20, verbose_eval=False)
    
    # # 訓練データに対する予測、正答率
    # y_pred = clf.predict(x_train)
    # acc_train = accuracy_score(y_true=y_train, y_pred=y_pred)
    # print(f"{itr:02}train_訓練データの正答率: {100 * acc_train:.2f} %")
    # scores_train.append(acc_train)  
    
    # 検証データに対する予測
    # accurasy_scoreするには0で丸め込みが必要（https://shiokoji11235.com/two_interface_of_lightgbm）
    y_pred = model.predict(x_valid)
    y_pred = y_pred.round(0)

    score_valid = accuracy_score(y_valid, y_pred)
    print(f"{itr:02}valid_accuracy_score: {score_valid:.4f} %")    

    # score_valid = log_loss(y_true=y_valid, y_pred=y_pred)
    # print(f"{itr:02}valid_logloss: {score_valid:.4f} %")    
    # score_valid = roc_auc_score(y_valid, y_pred)
    # print(f"{itr:02}valid_auc_score: {score_valid:.4f} %")    

    scores_valid.append(score_valid)    
    models.append(model)
    
    itr += 1

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006574 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
01valid_accuracy_score: 0.6982 %
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006321 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
02valid_accuracy_score: 0.6967 %
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006048 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
03valid_accuracy_score: 0.6992 %
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006161 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_c

In [111]:
# 各foldのスコアの平均
# print(np.mean(scores_train))
print(np.mean(scores_valid))

0.6990065576241803


## 推論

In [112]:
y_preds = np.zeros(len(x_test))

for model in models:
    y_pred = model.predict(x_test)
    y_pred = y_pred.round(0)
    y_preds += y_pred / len(models)
    
# print(y_test)
# print('***')
# print(y_preds)
# print('***')
# print(y_pred)

y_preds = y_preds.round(0)
score_test = accuracy_score(y_test, y_preds)
print(f"test_accuracy_score: {100 * score_test:.4f} %")    

test_accuracy_score: 71.9831 %


### input_01.csv
- 検証データに対する正答率：0.69899


### input_02.csv（class, main_category）
- 検証データに対する正答率：0.70896


### input_02.csv（class）
- 検証データに対する正答率：0.70765


### input_03.csv（class, yar/month/day）
- 検証データに対する正答率：0.7159


### input_04.csv
- ：71.78%
x_cols = ['class', 'country', 'period', 'n_words', 'log_goal', 'launched_year', 'launched_month', 'launched_day', 
         'name_vowel_ratio', 'Goal_1000', 'Goal_500', 'Goal_10', 'median_goal_Last_Week']

## テストデータに対する予測
本来はtrain/validではなくtrain/valid/testに分割し、testに対する予測を見るべき